## Creating Redshift Cluster using the AWS python SDK 

### Create clients for IAM, EC2, S3 and Redshift

In [1]:
import boto3
import configparser
import json
import pandas as pd

In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [3]:
import boto3


ec2 = boto3.resource('ec2',
                    region_name = 'us-west-2',
                    aws_access_key_id = config['IAM_ROLE']['KEY'],
                    aws_secret_access_key = config['IAM_ROLE']['SECRET']
                    )

iam = boto3.client('iam',
                    region_name = 'us-west-2',
                    aws_access_key_id = config['IAM_ROLE']['KEY'],
                    aws_secret_access_key = config['IAM_ROLE']['SECRET'])

redshift = boto3.client('redshift',
                       region_name = 'us-west-2',
                       aws_access_key_id = config['IAM_ROLE']['KEY'],
                       aws_secret_access_key = config['IAM_ROLE']['SECRET'])


## STEP 1: IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [4]:
from botocore.exceptions import ClientError
DWH_IAM_ROLE_NAME = config['IAM_ROLE']['DWH_IAM_ROLE_NAME']
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a new IAM Role
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::016388214636:role/dwhRole


## Step 2: Redshift Cluster

In [5]:
DWH_CLUSTER_TYPE = config['DWH']['DWH_CLUSTER_TYPE']
DWH_NUM_NODES = config['DWH']['DWH_NUM_NODES']
DWH_NODE_TYPE = config['DWH']['DWH_NODE_TYPE']
DWH_CLUSTER_IDENTIFIER = config['DWH']['DWH_CLUSTER_IDENTIFIER']
DWH_DB = config['DWH']['DWH_DB']
DWH_DB_USER = config['DWH']['DWH_DB_USER']
DWH_DB_PASSWORD = config['DWH']['DWH_DB_PASSWORD']
DWH_PORT = config['DWH']['DWH_PORT']
DWH_IAM_ROLE_NAME = config['DWH']['DWH_IAM_ROLE_NAME']

try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

In [6]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,creating
3,MasterUsername,dwhuser
4,DBName,sparkify_dwh
5,VpcId,vpc-428eb13a
6,NumberOfNodes,4


## Step 3: Open an incoming TCP port to access the cluster endpoint 

In [9]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-9792fea2')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


## Step 4: Clean up all resources

In [ ]:
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

In [ ]:
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)